In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import datetime

drive.mount('/content/drive')

path = '/content/drive/MyDrive/database (1).csv'

import pandas as pd
df = pd.read_csv(path)
df

Mounted at /content/drive


,Year,Ceremony,Award,FirstName,LastName,Film
0,1927/1928,1,Actor,Richard,Barthelmess,The Noose
1,1927/1928,1,Actor,Emil,Jannings,The Last Command
2,1927/1928,1,Actress,Louise,Dresser,A Ship Comes In
3,1927/1928,1,Actress,Janet,Gaynor,7th Heaven
4,1927/1928,1,Actress,Gloria,Swanson,Sadie Thompson
...,...,...,...,...,...,...
9959,2015,88,Writing (Original Screenplay),Spotlight,NaN,Written by Josh Singer & Tom McCarthy
9960,2015,88,Writing (Original Screenplay),Straight,Outta,Screenplay by Jonathan Herman and Andrea Berlo...
9961,2015,88,Jean Hersholt Humanitarian Award,Debbie,Reynolds,NaN
9962,2015,88,Honorary Award,Spike,Lee,NaN


In [ ]:
import pandas as pd
import re

# Regex patterns for names
full_name_pattern = re.compile(r'^[A-Z][a-z]+\s[A-Z][a-z]+$', re.IGNORECASE)  # "John Doe"
single_name_pattern = re.compile(r'^[A-Z][a-z]+$', re.IGNORECASE)  # "John" or "Doe"

# Common non-name values (race, gender labels, etc.)
non_name_values = {"male", "female", "nonbinary", "black", "white", "asian", "hispanic", "latino", "native"}

# Function to refine name column detection
def find_name_columns(df):
    name_columns = []

    for column in df.columns:
        values = df[column].astype(str)

        match_count_full = values.apply(lambda x: bool(full_name_pattern.match(x))).sum()
        match_count_single = values.apply(lambda x: bool(single_name_pattern.match(x))).sum()
        non_name_count = values.apply(lambda x: x.lower() in non_name_values).sum()

        # Check if values are consistently short (likely a name, not descriptive text)
        avg_word_count = values.apply(lambda x: len(str(x).split())).mean()

        # Consider a column a "name column" if:
        # - At least 70% of values match name patterns
        # - Less than 10% match race/gender labels
        # - Word count average is **≤ 2** (filters out descriptive categorical data)
        if max(match_count_full, match_count_single) / len(values) > 0.7 and non_name_count / len(values) < 0.1 and avg_word_count <= 2:
            name_columns.append(column)

    return name_columns

# Identify columns containing names
name_columns = find_name_columns(df)

# Print results
if name_columns:
    print(f"✅ Name columns found: {name_columns}")
else:
    print("❌ No name columns detected.")

✅ Name columns found: ['FirstName ', 'LastName']


In [ ]:
import pandas as pd


# Function to extract only the first letter of names
def first_letter_only(name):
    return name[0] if isinstance(name, str) and name.strip() != "" else ""  # Ensures valid input

# Apply transformation **only to first 50 rows**
df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)

# Print results
print(df.head(50))  # Show first 50 modified rows

         Year  Ceremony                         Award FirstName  LastName  \
0   1927/1928         1                         Actor          R        B   
1   1927/1928         1                         Actor          E        J   
2   1927/1928         1                       Actress          L        D   
3   1927/1928         1                       Actress          J        G   
4   1927/1928         1                       Actress          G        S   
5   1927/1928         1                 Art Direction          R        G   
6   1927/1928         1                 Art Direction          W        C   
7   1927/1928         1                 Art Direction          H        O   
8   1927/1928         1                Cinematography          G        B   
9   1927/1928         1                Cinematography          C        R   
10  1927/1928         1                Cinematography          K        S   
11  1927/1928         1    Directing (Comedy Picture)          L        M   

<ipython-input-3-1033100105>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)


In [ ]:
import pandas as pd

# Function to classify individual values
def classify_name(value):
    value = str(value).strip()
    if len(value) == 1:  # Single character = Initial
        return "Initial"
    elif len(value) > 1:  # More than one character = Full Name
        return "Full Name"
    else:
        return "Unknown"


# Apply classification function to each value
for col in name_columns:
    df[f"{col} Type"] = df[col].apply(classify_name)

# Count full names and initials
for col in name_columns:
    full_names_count = (df[f"{col} Type"] == "Full Name").sum()
    initials_count = (df[f"{col} Type"] == "Initial").sum()

    # Print results
    print(f"❌ {col}: {full_names_count} full names, {initials_count} initials, Data is inconsistent. Suggest getting the full name for values with just initials.")

❌ FirstName : 9676 full names, 288 initials, Data is inconsistent. Suggest getting the full name for values with just initials.
❌ LastName: 9811 full names, 150 initials, Data is inconsistent. Suggest getting the full name for values with just initials.
